<a href="https://www.kaggle.com/ridhomuhammad/obj-detection-ssd-v1?scriptVersionId=84864737" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
!mkdir Tensorflow
!cp -r ./models ./Tensorflow/
!mkdir Tensorflow/workspace 
!cp -r ../input/d/ridhomuhammad/sibi-sign-language/NEW_AL ./Tensorflow/workspace



In [ ]:
!mkdir ./Tensorflow/workspace/fine_tuned_model
!mkdir ./Tensorflow/workspace/fine_tuned_model/saved_model
!mkdir ./Tensorflow/workspace/exported_models

In [ ]:
# !zip -r ckp_ridho.zip !ls ./Tensorflow/workspace/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
!ls ./

In [ ]:
from IPython.display import FileLink
FileLink(r'./roboflow.zip')

In [ ]:
!cp -r ../input/d/ridhomuhammad/sibi-sign-language/anot ./Tensorflow/workspace/annotations/

In [ ]:
%cd Tensorflow/workspace/
!curl -L "https://app.roboflow.com/ds/6LLb0DMJ0O?key=Yl3Nl86oBD" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd ../..

In [ ]:
!rm -r ./models

In [ ]:
%%bash
cd ./Tensorflow/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
!ls ./Tensorflow/workspace/test

In [ ]:
test_record = './Tensorflow/workspace/test/A.tfrecord'
train_record = './Tensorflow/workspace/train/A.tfrecord'
label_map_pbtxt_fname = './Tensorflow/workspace/train/A_label_map.pbtxt'

In [ ]:
MODELS_CONFIG = {
    'ssd-mobilenet' : {
        'model_name': 'ssd_mobilenet_v2',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
        'batch_size' : 16
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    }
}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'ssd-mobilenet'

num_steps = 4000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size']

In [ ]:
%cd ./Tensorflow/workspace
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [ ]:
!ls

In [ ]:
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
# !rm ./Tensorflow/workspace/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config.1
# !rm ./Tensorflow/workspace/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config.2

In [ ]:
%cd ../..

In [ ]:
pipeline_fname = './Tensorflow/workspace/' + base_pipeline_file
fine_tune_checkpoint = './Tensorflow/workspace/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
import re

# %cd ./Tensorflow/workspace/
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
!ls

In [ ]:
pipeline_file = './pipeline_file.config'
model_dir = './Tensorflow/workspace/models'

In [ ]:
!cp ./pipeline_file.config ./Tensorflow/workspace/

In [ ]:
!python ./Tensorflow/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps=4000 \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=500

In [ ]:
!ls ./Tensorflow/workspace/models

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import config_util

In [ ]:
!ls ./Tensorflow/workspace/

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap('./Tensorflow/workspace/train/A_label_map.pbtxt')
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file('./pipeline_file.config')
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join("./Tensorflow/workspace/models/", 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


In [ ]:
# frame = cv2.imread('./Tensorflow/workspace/NEW_AL/NEW_AL/test/A3_62.JPG')
# image_np = np.array(frame)
# input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
# # print(input_tensor.shape)
# # det = detect_fn(input_tensor)
# img, shape = detection_model.preprocess(input_tensor)
# det = detection_model.predict(img, shape)
# dets = detection_model.postprocess(det, shape)

# num_detections = int(dets.pop('num_detections'))
# detections = {key: value[0, :num_detections].numpy()
#               for key, value in dets.items()}
# dets['num_detections'] = num_detections
# # detection_classes should be ints.
# # dets['detection_classes'] = dets['detection_classes'].astype(np.int64)
# # label_id_offset = 1
# # image_np_with_detections = image_np.copy()
# print(dets['num_detections'])


In [ ]:
!ls Tensorflow/workspace/fin

In [ ]:
#run conversion script
import re
import numpy as np                

output_directory = '/Tensorflow/workspace/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = './Tensorflow/workspace/models'
print(last_model_path)
!python ./Tensorflow/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [ ]:
!ls ./Tensorflow/workspace/fine_tuned_model/saved_model/

In [ ]:
saved_tflite_inference ="./Tensorflow/workspace/exported_models/tflite_inference/saved_model"

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 320, 320, 3)
      yield [data.astype(np.float32)]

In [ ]:
import numpy as np
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(saved_tflite_inference)
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.uint8  # or tf.uint8
converter.inference_output_type = tf.uint8  # or tf.uint8
tflite_quant_model = converter.convert()

In [ ]:
tf_lite_model_path_without_metadata = "./Tensorflow/workspace/model_output_tflite/tflite/detect_without_metadata.tflite"

In [ ]:
!mkdir ./Tensorflow/workspace/model_output_tflite/tflite

In [ ]:
with tf.io.gfile.GFile(tf_lite_model_path_without_metadata, 'wb') as f:
  f.write(tflite_quant_model)

In [ ]:
!pip install --upgrade 'setuptools<42'

In [ ]:
!pip install 'pybind11>=2.4'
!pip install tflite-support

In [ ]:

from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = tf_lite_model_path_without_metadata
_LABEL_FILE = "./Tensorflow/workspace/train/A_label_map.pbtxt"
_SAVE_TO_PATH = "./Tensorflow/workspace/model_output_tflite/tflite/detect_with_metadata.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

In [ ]:
%cd tflite

In [ ]:
!cp ./Tensorflow/workspace/model_output_tflite/tflite/detect_with_metadata.tflite ../input

In [ ]:
%cd ./Tensorflow/workspace/model_output_tflite/

In [ ]:
!zip -r Tensorflow.zip ./Tensorflow/

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'./tf.zip')